In [12]:
import os
import json
import sys
from autocomplete import Autocomplete, PY_LANGUAGE, Parser
import argparse
from tqdm import tqdm
from extract_api_doc import *
from extract_csn_python_api import count_jsonl, read_jsonl, write_jsonl, write_jsonl_append
import functools

# inp_path = "/Users/zkcpku/Documents/seke/mywork/dependency/cloudbrain/dataset/CSN_valid_np_pd.only_search.jsonl"
inp_path = "/Users/zkcpku/Documents/seke/mywork/APItoolformer/cloudbrain/dataset/CSN_valid_np_pd328.only_search.jsonl"

parser = Parser()
parser.set_language(PY_LANGUAGE)


In [13]:
def get_apiend_pos(APISearch_pos, code_string):
    APISearch_end_pos = APISearch_pos[2]
    apiend_pos = code_string[APISearch_end_pos:].find("*/")
    if apiend_pos == -1:
        assert False
        return None
    apiend_pos += APISearch_end_pos
    return apiend_pos

In [14]:
def string_find_all(code_string, s):
    pos = 0
    pos_list = []
    while True:
        pos = code_string.find(s, pos)
        if pos == -1:
            break
        pos_list.append(pos)
        pos += len(s)
    return pos_list
def add_apisearch_rst(code_string):
    # code_string = code_string.replace("*/","*/ ")
    code = bytes(code_string, "utf-8")
    tree = parser.parse(code)
    completer = Autocomplete(tree, code, len(code))
    api_calls = completer.extract_used_api()
    api_end_pos = string_find_all(code_string, "*/")
    api_calls = sorted(api_calls, key=lambda x:x[2])

    def get_close_api_for_each(api_end_poses, extract_apies):
        close_api_for_each = []
        for i in range(len(api_end_poses)):
            this_close_api = "None"
            for j in range(len(extract_apies)):
                if api_end_poses[i] < extract_apies[j][2]:
                    this_close_api = extract_apies[j][0]
                    break
            close_api_for_each.append(this_close_api)
        return close_api_for_each
    def clean_close_api(origin_api):
        # if "(" in origin_api:
        #     origin_api = origin_api.split("(")[0]
        return origin_api
    # print(api_calls)
    SearchRst_apis = get_close_api_for_each(api_end_pos, api_calls)
    SearchRst_apis = [clean_close_api(e) for e in SearchRst_apis]
    return SearchRst_apis
    
    def add_into_code_string(code_string, api_end_pos, SearchRst_apis):
        split_code_string_pos = api_end_pos
        split_code_string_pos.append(len(code_string))
        split_code_string = []
        for i in range(len(split_code_string_pos)):
            if i == 0:
                split_code_string.append(code_string[:split_code_string_pos[i]])
            else:
                split_code_string.append(code_string[split_code_string_pos[i-1]:split_code_string_pos[i]])
        new_code_string = ""
        for i in range(len(split_code_string)):
            new_code_string += split_code_string[i]
            if i < len(split_code_string) - 1:
                new_code_string += f"->{SearchRst_apis[i]}"
        return new_code_string
    return add_into_code_string(code_string, api_end_pos, SearchRst_apis)

In [15]:
inp_data = read_jsonl(inp_path)

In [17]:
output_path = "/Users/zkcpku/Documents/seke/mywork/dependency/cloudbrain/dataset/CSN_valid_np_pd.with_rst.jsonl"
all_apis = []
for each_obj in tqdm(inp_data):
    each_obj["without_rst"] = each_obj['func_code_string']
    all_apis.append(add_apisearch_rst(each_obj['func_code_string']))
    # each_obj['func_code_string'] = add_apisearch_rst(each_obj['func_code_string'])
    # write_jsonl_append(output_path, each_obj)

19210it [00:34, 553.63it/s]


In [18]:
all_apis[0]

['np.array', 'None']

In [19]:
all_apis = [ee for e in all_apis for ee in e]

In [20]:
all_apis = [e for e in all_apis if e and e != "None"]

In [21]:
len(all_apis)

59101

In [23]:
modules = [e.split(".")[0] for e in all_apis]

In [24]:
from collections import Counter

In [26]:
keys = Counter(modules)

In [27]:
keys = sorted(keys.items(), key=lambda x:x[1], reverse=True)

In [29]:
3607+1581

5188

In [28]:
keys

[('np', 29383),
 ('numpy', 3607),
 ('APISearch', 3509),
 ('pd', 1581),
 ('self', 993),
 ('len', 637),
 ('_np', 441),
 ('max', 271),
 ('plt', 226),
 ('int', 199),
 ('pandas', 187),
 ('sum', 177),
 ('scipy', 165),
 ('range', 163),
 ('isinstance', 162),
 ('list', 144),
 ('min', 141),
 ('os', 141),
 ('zip', 137),
 ('sl', 134),
 ('my_einsum', 126),
 ('abs', 119),
 ('utils', 110),
 ('tf', 106),
 ('sparse', 97),
 ('math', 92),
 ('sp', 92),
 ('df', 87),
 ('str', 86),
 ('set', 81),
 ('sorted', 80),
 ('data', 80),
 ('xrange', 75),
 ('stats', 75),
 ('float', 69),
 ('random', 69),
 ('ValueError', 66),
 ('getattr', 66),
 ('norm', 61),
 ('cv2', 61),
 ('x', 60),
 ('enumerate', 58),
 ('onp', 58),
 ('tuple', 57),
 ('sin', 57),
 ('dict', 54),
 ('ax', 53),
 ('copy', 53),
 ('reshape', 52),
 ('slice', 52),
 ('cos', 52),
 ('time', 49),
 ('map', 49),
 ('type', 48),
 ('sqrt', 48),
 ('torch', 47),
 ('datetime', 43),
 ('linalg', 42),
 ('hp', 41),
 ('hasattr', 41),
 ('mx', 41),
 ('array', 36),
 ('chaospy', 36),
